### Info
Author: S. Bakker

Date: January 2023

#### Importing packages and setting project data based on DEM

In [2]:
import os
import gzip
import shutil
import rasterio
from rasterio.merge import merge
from rasterio.plot import show
import numpy as np
from rasterio.warp import reproject, Resampling, calculate_default_transform

NO_DATA = -9999

In [3]:
WD = 'C:/Users/siepb/Documents/Studie_Hydrology/ThesisGeul/Raw_DEM/DEM_HighRes/Germany/'
WD_XYZ = f'{WD}xyz/'
WD_TIF = f'{WD}tif/'

small_tiles = os.listdir(WD_XYZ)
small_tiles = ([s.strip('.xyz.gz') for s in small_tiles])

In [55]:
for tile in small_tiles:
    with gzip.open(f'{WD_XYZ}{tile}.xyz.gz', 'rb') as open_xyz:
        with rasterio.open(open_xyz) as open_raster:
            with rasterio.open(f'{WD_TIF}{tile}.tif', 
                               'w',
                                driver = 'GTiff',
                               dtype = 'float32',
                               width = open_raster.width,
                               height = open_raster.height,
                               count=1,
                               crs = open_raster.crs,
                               transform = open_raster.transform) as write_xyz:
                write_xyz.write(open_raster.read(1), indexes = 1)

In [2]:
# Creating an initial file to add the other tiles to
with rasterio.open(f'{WD_TIF}{small_tiles[0]}.tif', 'r') as ras:
    array = ras.read(1)
    with rasterio.open(f'{WD}GER_mergedfile.tif', 
                       'w', 
                       driver = 'GTiff',
                       dtype = 'float32',
                       height = ras.height,
                       width = ras.width,
                       count= 1,
                       crs = ras.crs,
                       transform = ras.transform,
                      ) as dst:
        dst.write(array, indexes = 1)

        
# |Adding the tiles one by one (highly inefficient so takes a lot of time, but effective)      
for i in range(1, (len(small_tiles))):
    with rasterio.open(f'{WD}GER_mergedfile.tif', 'r') as ras_1:
        with rasterio.open(f'{WD_TIF}{small_tiles[i]}.tif', 'r') as ras_2:
            merged, trans = merge([ras_1, ras_2], nodata = NO_DATA)
            
    with rasterio.open(f'{WD}GER_mergedfile.tif', 
                       'w', 
                       driver = 'GTiff',
                       dtype = 'float32',
                       height = merged.shape[1],
                       width = merged.shape[2],
                       count= 1,
                       crs = ras_1.crs,
                       transform = trans,
                       nodata = NO_DATA
                      ) as dst:
        dst.write(merged)

NameError: name 'rasterio' is not defined